<a href="https://colab.research.google.com/github/ykitaguchi77/GAN/blob/master/ChatGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
from google.colab import drive
drive.mount("/content/drive")

!pip install openai --q
!pip install --upgrade deepl --q
import openai
import deepl
import textwrap
import pandas as pd

with open("/content/drive/MyDrive/Deep_learning/api.txt") as file:
    #text = file.read()
    i=1
    key = []
    while True:
        include_break_line = file.readline() #改行が含まれた行
        line = include_break_line.rstrip() #改行を取り除く
        if line: #keyの読み込み
            #print(f'{i}行目：{line}')
            key.append(line)
            i += 1
        else:
            break

# APIキーの準備
#ngrok_aceess_token = key[5]
openai.api_key = key[3]
deepl_auth_key = key[1]

translator = deepl.Translator(deepl_auth_key)

k = 0

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [108]:
#@title #**ChatGPT configulation**
question_translate_English= True #@param {type:"boolean"}
answer_Japanese= False #@param {type:"boolean"}
reset= False #@param {type:"boolean"}
show_all = True #@param {type:"boolean"}
arrangement = True #@param {type:"boolean"}



if "text_input" not in globals():
    text_input = []
if reset == True:
    text_input = []
    k = 0


def wrap(sentence):
    wrap_list =  textwrap.wrap(sentence, 120)
    output = '\n'.join(wrap_list)
    return(output)


def interference(question, k):
    question = question.replace('\n', '') #前後の空白と改行を削除

    if question_translate_English == True:
        question_translated = translator.translate_text(question, target_lang="EN-US").text
        question = question_translated
    else:
        question = question

    prompt = "Q:" + question +"\nA:"

    text_input.append(question+"\nA:")

    prompt =  ",".join(map(str, text_input)) # aaa,bbb,100,200,300 

    response = openai.Completion.create(
      model="text-davinci-003",
      prompt=prompt,
      temperature=0.5,
      max_tokens=1024,
      #top_p=1,
      frequency_penalty=0.0,
      presence_penalty=0.0,
      stop=None)

    answer = response['choices'][0]['text']
    text_input.append(answer)

    if answer_Japanese == True:
        answer = translator.translate_text(response['choices'][0]['text'], target_lang="JA").text
    else:
        answer = answer

    output(text_input, show_all=show_all, arrangement=arrangement)
    # print(f"{prompt}")
    # print(f"{wrap(answer)} \n")
    #print(text_input)


def output(text_input, show_all = True, arrangement = False):
    if arrangement == True:
        if show_all == False:
            print(f"Q: {wrap(text_input[-2]).rstrip('A:')} \n")
            print(f"A: {wrap(text_input[-1])} \n")

        elif show_all == True:
            for i, text in enumerate(text_input):
                if i % 2 == 0:
                    print(f"Q: {wrap(text).rstrip('A:')} \n")
                else:
                    print(f"A: {wrap(text)} \n")

    elif arrangement == False:
        if show_all == False:
            print(f"Q: {text_input[-2].rstrip('A:')} \n")
            print(f"{text_input[-1]} \n")

        elif show_all == True:
            for i, text in enumerate(text_input):
                if i % 2 == 0:
                    print(f"Q: {text.rstrip('A:')} \n")
                else:
                    print(f"{text} \n")




#**Interference**

In [ ]:
question = '''
The first paragraph begins with "Corneal diseases are a common and debilitating condition that affects the transparency of the clear front surface of the eye."
The paragrapy ends with "early identification of corneal diseases is essential to preserve vision."
論理性が保たれるように間の文を埋めて下さい。
全部で4つ〜8つの文章にして下さい。

'''
interference(question)

In [ ]:
output(text_input, show_all = False, arrangement = False)